# Pretrained One-vs-Rest Classifier

This notebook implements an one-vs-rest classifier that fine-tunes several BERT models to tell if a sentence contains problematic metaphors. It starts out from our BERT model pretrained on a large corpus of data.

<div hidden>
TODO: add extend data3/data.json with better data in the same format that actually makes sense.
</div>

## Imports and Setup

In [1]:
%pip install transformers -Uqq
%pip install sklearn -Uqq
%pip install datasets -Uqq
%pip install torch -Uqq
%pip install numpy -Uqq
%pip install evaluate -Uqq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import evaluate
import numpy as np
import torch
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
)
import os
from itertools import product

## Loading Dataset

In [4]:
dataset = load_dataset("json", data_files="data/data.json", field="data")
dataset

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-e7751a31f3f17887/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'agency', 'humanComparison', 'hyperbole', 'historyComparison', 'unjustClaims', 'deepSounding', 'sceptics', 'deEmphasize', 'performanceNumber', 'inscrutable'],
        num_rows: 329
    })
})

In [5]:
dataset["train"][0:3]

{'text': ['A new vision of artificial intelligence for the people',
  'The gig workers fighting back against the algorithms',
  'How the AI industry profits from catastrophe'],
 'agency': [False, True, False],
 'humanComparison': [False, True, False],
 'hyperbole': [False, True, True],
 'historyComparison': [False, False, False],
 'unjustClaims': [False, False, False],
 'deepSounding': [False, False, False],
 'sceptics': [False, False, False],
 'deEmphasize': [False, False, False],
 'performanceNumber': [False, False, False],
 'inscrutable': [False, False, False]}

In [6]:
labels = [label for label in dataset["train"].features.keys() if label not in ["text"]]

num_epochs = {
    "agency": 10,
    "humanComparison": 2,
    "hyperbole": 2,
    "historyComparison": 2,
    "unjustClaims": 5,
    "deepSounding": 2,
    "sceptics": 2,
    "deEmphasize": 7,
    "performanceNumber": 2,
    "inscrutable": 2,
}

labels

['agency',
 'humanComparison',
 'hyperbole',
 'historyComparison',
 'unjustClaims',
 'deepSounding',
 'sceptics',
 'deEmphasize',
 'performanceNumber',
 'inscrutable']

## Preprocess Data, Create Train/Test Split

In [7]:
processed_dataset = {}
for label in labels:
    projected_dataset = (
        dataset["train"]
        .map(remove_columns=[l for l in labels if l != label])
        .rename_column(label, "labels")
        .class_encode_column("labels")
    )
    processed_dataset[label] = projected_dataset.train_test_split(
        test_size=0.2, stratify_by_column="labels"
    )
    # print(f"{label}:\n\t{processed_dataset[label]['test'][0:3]}\n")

processed_dataset

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e7751a31f3f17887/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-604182da521c301e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e7751a31f3f17887/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-8d3dee6ea1033dcd.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e7751a31f3f17887/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-839d38437908d34a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e7751a31f3f17887/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-fc7d1db672f398d9.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e7751a31f3f17887/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-36f8b4671a097526.arrow


{'agency': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'humanComparison': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'hyperbole': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'historyComparison': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'unjustClaims': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     t

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
tokenized_dataset = {
    k: ds.map(
        preprocess_data,
        remove_columns="text",
        batched=True,
    )
    for k, ds in processed_dataset.items()
}

Map:   0%|          | 0/263 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/263 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/263 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/263 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/263 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

### Verify dataset

In [10]:
example = tokenized_dataset["agency"]["train"][0]
print(example.keys())

dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [11]:
tokenizer.decode(example["input_ids"])

'[CLS] Big Tech ’ s guide to talking about AI ethics [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [12]:
example["labels"]

0

## Load Pre-Trained Model

In [13]:
# use_fast uses fast tokenizers backed by rust. Remove it if it causes errors
# model = AutoModelForSequenceClassification.from_pretrained(
    # "bert-base-cased",
    # num_labels=2,
# )

### Verify data-model interaction

In [14]:
# forward pass
# outputs = model(
# input_ids=tokenized_dataset[labels[0]]["train"]["input_ids"][0],
# labels=tokenized_dataset[labels[0]]["train"][0]["labels"],
# )
# outputs

## Define Metrics

In [15]:
metrics = {
    "accuracy": evaluate.load("accuracy"),
    "presicion": evaluate.load("precision"),
    "recall": evaluate.load("recall"),
    "f1": evaluate.load("f1"),
}

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    values = {}
    
    for name, metric in metrics.items():
        result = metric.compute(predictions=predictions, references=labels)
        for val in result.values() if isinstance(result, dict) else [result]:
            values[name] = val

    return values

In [17]:
class HighPrecisionTrainer(Trainer):
    """A trainer class, which computes loss based on a weighted MSE, where the error for the positive labels is
    weighted more than the error for the negative labels, leading to a higher precision
    """

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        label_mask = torch.FloatTensor(1, 2).cuda().zero_()
        label_mask[0, labels[0]] = 1
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss (suppose one has 3 labels with different weights)
        loss = torch.sum(torch.tensor([[1, 300]]).cuda() * ((logits - label_mask) ** 2))
        return (loss, outputs) if return_outputs else loss

## Train the Model

In [23]:
  # TODO: increase if we have more data
metric_name = "recall"

In [ ]:
for model_name, weight_decay in product(['bert-base-cased', 'xt0r3/aihype_bert_fine_tune'], [0.0, 0.01]):
    for i in range(0, 5):
        batch_size = 2 ** i
        print(f'batch size: {2 ** i}\nmodel: {model_name}\nweight decay: {weight_decay}')

        for label in ['agency']:  # labels:
            print(f"training model for {label}")

            model = AutoModelForSequenceClassification.from_pretrained(
                model_name,
                num_labels=2,
            )

            training_args = TrainingArguments(
                f"{model_name.replace('/', '_')}_{label}-vs-rest",
                evaluation_strategy="epoch",
                save_strategy="epoch",
                learning_rate=2e-5,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                num_train_epochs=num_epochs[label] + i * 2,
                weight_decay=weight_decay,
                report_to="none",
                load_best_model_at_end=True,
                metric_for_best_model=metric_name,
                save_total_limit = 1,
                # push_to_hub=True,  # TODO: enable once model seems good
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_dataset[label]["train"],
                eval_dataset=tokenized_dataset[label]["test"],
                compute_metrics=compute_metrics,
            )

            trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/pytorch_model.b

batch size: 1
model: bert-base-cased
weight decay: 0.0
training model for agency


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss,Accuracy,Presicion,Recall,F1
1,No log,0.882871,0.787879,0.000000,0.000000,0.000000
2,1.018600,1.060922,0.787879,0.000000,0.000000,0.000000
3,1.018600,1.516878,0.772727,0.333333,0.071429,0.117647
4,0.303700,1.769249,0.772727,0.000000,0.000000,0.000000
5,0.303700,1.712236,0.787879,0.500000,0.285714,0.363636
6,0.039900,1.835033,0.787879,0.500000,0.214286,0.300000
7,0.039900,1.889408,0.787879,0.500000,0.214286,0.300000
8,0.000100,1.885001,0.787879,0.500000,0.214286,0.300000
9,0.000100,1.897952,0.787879,0.500000,0.214286,0.300000
10,0.000100,1.900281,0.787879,0.500000,0.214286,0.300000


***** Running Evaluation *****
  Num examples = 66
  Batch size = 1
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to bert-base-cased_agency-vs-rest/checkpoint-263
Configuration saved in bert-base-cased_agency-vs-rest/checkpoint-263/config.json
Model weights saved in bert-base-cased_agency-vs-rest/checkpoint-263/pytorch_model.bin
Deleting older checkpoint [bert-base-cased_agency-vs-rest/checkpoint-526] due to args.save_total_limit
Deleting older checkpoint [bert-base-cased_agency-vs-rest/checkpoint-789] due to args.save_total_limit
Deleting older checkpoint [bert-base-cased_agency-vs-rest/checkpoint-1052] due to args.save_total_limit
Deleting older checkpoint [bert-base-cased_agency-vs-rest/checkpoint-1315] due to args.save

batch size: 2
model: bert-base-cased
weight decay: 0.0
training model for agency


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss


## Upload the Model

In [ ]:
# agency-vs-rest/checkpoint-263: 0.75 precision, 0.85 recall
#

In [ ]:
# trainer.push_to_hub()